# **Deep Learning - Rendu n°2 : Cas d'études**

Elyes KHALFALLAH - 5230635

13/12/2024

---


## **1 :** Jeu de données


### **1.2 :** Quelques statistiques descriptives


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [ ]:
# Chargement des fichiers CSV
train_df = pd.read_csv("train.csv", sep="\t")
test_df = pd.read_csv("test.csv", sep="\t")


In [ ]:
# Afficher les quelques premieres lignes
train_df.head()


In [ ]:
# Obtenir les types des données de chaque colonnes
train_df.info()


- `float64(9)`, `int64(4)`, nous avons 13 données de type quantitatives
- `object(7)`, nous avons 7 données de type descriptives


---


Commençons par approfondir notre analyse : examinons quelques statistiques descriptives sur les variables quantitatives. Nous observerons aussi la distribution des genres et sous-genres afin de mieux comprendre la répartition des pistes dans notre ensemble de données. Ce premier aperçu nous aidera à saisir les principales tendances et à identifier d’éventuels déséquilibres dans les catégories.


In [ ]:
# Obtenir des statistiques descriptives des colonnes numériques
train_df.describe()


In [ ]:
# Distribution des genres musicaux des morceaux regardés
genre_counts = train_df["playlist_genre"].value_counts()
print(genre_counts)


In [ ]:
# Distribution des sous-genres musicaux des morceaux regardés
subgenre_counts = train_df["playlist_subgenre"].value_counts()
print(subgenre_counts)


In [ ]:
# Histogrammes des distributions des variables numériques

num_cols = [
    "track_popularity",
    "danceability",
    "energy",
    "key",
    "loudness",
    "mode",
    "speechiness",
    "acousticness",
    "instrumentalness",
    "liveness",
    "valence",
    "tempo",
    "duration_ms",
]

train_df[num_cols].hist(figsize=(16, 16))
plt.tight_layout()
plt.show()


---


Avant de poursuivre, nous allons explorer visuellement plusieurs aspects clés de notre jeu de données. Nous commencerons par un histogramme illustrant la distribution des genres, puis nous examinerons la matrice de corrélation afin d’identifier les relations entre les variables quantitatives. Nous calculerons également la popularité moyenne des pistes par genre et, enfin, nous visualiserons la relation entre la dansabilité et l’énergie, réparties par genre, pour approfondir notre compréhension des caractéristiques musicales.


In [ ]:
# Histogramme pour la répartition des genres

plt.figure(figsize=(10, 6))
ax = genre_counts.plot(kind="bar", color="skyblue", edgecolor="black")
plt.title("Distribution des genres de playlist", fontsize=16)
plt.xlabel("Genre", fontsize=14)
plt.ylabel("Nombre de pistes", fontsize=14)
plt.grid(True, which="both", linestyle="--", linewidth=1, alpha=0.7)

# Le bout de code suivant, gérant l'affichage des valeurs sur les barres a été généré avec ChatGPT :
# Affichage des valeurs au-dessus des barres
for p in ax.patches:
    height = p.get_height()
    plt.text(
        p.get_x() + p.get_width() / 2,  # Position en x (centre de la barre)
        height,  # Position en y (hauteur de la barre)
        f"{int(height)}",  # Texte à afficher
        ha="center",  # Alignement horizontal
        va="bottom",  # Alignement vertical
        fontsize=12,
        color="black",
    )


plt.tight_layout()
plt.show()


In [ ]:
# Affichage d'une matrice de corrélation entre les variables numériques
plt.figure(figsize=(12, 10))
corr_matrix = train_df[num_cols].corr()
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm")
plt.title("Matrice de corrélation des variables numériques")
plt.show()


In [ ]:
# Calcule la popularité moyenne des pistes pour chaque genre de playlist
popularity_by_genre = train_df.groupby("playlist_genre")["track_popularity"].mean()
print(popularity_by_genre)


In [ ]:
# Nuage de points montrant la relation entre la dansabilité, l'énergie et le genre de playlist
plt.figure(figsize=(10, 6))
sns.scatterplot(data=train_df, x="danceability", y="energy", hue="playlist_genre")
plt.title("Relation entre la dansabilité et l'énergie par genre")
plt.show()


Ce code effectue une exploration préliminaire des données : il charge les fichiers d’entraînement et de test, examine la structure des données (types de variables, dimensions), calcule des statistiques descriptives, puis visualise les distributions, la répartition des genres, et les relations entre variables numériques. Ces analyses offrent une première compréhension de l’ensemble de données et posent les bases pour des traitements et modèles plus avancés.


---


### **1.3 :** Application d'algorithmes standards


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [ ]:
# Chargement des données d'entraînement
train_df = pd.read_csv("train.csv", sep="\t")


In [ ]:
# Liste des colonnes quantitatives
quantitative_columns = [
    "track_popularity",
    "danceability",
    "energy",
    "key",
    "loudness",
    "mode",
    "speechiness",
    "acousticness",
    "instrumentalness",
    "liveness",
    "valence",
    "tempo",
    "duration_ms",
]

# Sélection des features (X)
X = train_df[quantitative_columns]


In [ ]:
# Variable cible
y = train_df["playlist_genre"]

# Encodage des labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


In [ ]:
# Division des données en ensembles d'entraînement (80%) et de test (20%).
# `random_state` fixe la graine pour obtenir une division reproductible.
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42
)


In [ ]:
# Initialisation d'un scaler pour standardiser les données (moyenne = 0, écart-type = 1).
scaler = StandardScaler()

# Ajustement du scaler sur les données d'entraînement et transformation des données d'entraînement.
X_train_scaled = scaler.fit_transform(X_train)

# Transformation des données de test en utilisant les paramètres appris sur l'ensemble d'entraînement.
X_test_scaled = scaler.transform(X_test)


#### SVM


In [ ]:
# Initialisation d'un classificateur SVM (Support Vector Machine).
svm_classifier = SVC()

# Entraînement du SVM sur les données d'entraînement standardisées.
svm_classifier.fit(X_train_scaled, y_train)

# Prédiction des étiquettes pour les données de test standardisées.
y_pred_svm = svm_classifier.predict(X_test_scaled)

# Calcul de la précision du SVM sur l'ensemble de test.
accuracy_svm = accuracy_score(y_test, y_pred_svm)

# Affichage de la précision obtenue.
print(f"Accuracy du SVM : {accuracy_svm:.2f}")


#### Arbre de décision


In [ ]:
# Initialisation d'un classificateur Arbre de Décision.
dt_classifier = DecisionTreeClassifier()

# Entraînement de l'Arbre de Décision sur les données d'entraînement (non standardisées).
dt_classifier.fit(X_train, y_train)

# Prédiction des étiquettes pour les données de test.
y_pred_dt = dt_classifier.predict(X_test)

# Calcul de la précision de l'Arbre de Décision sur l'ensemble de test.
accuracy_dt = accuracy_score(y_test, y_pred_dt)

# Affichage de la précision obtenue.
print(f"Accuracy de l'Arbre de Décision : {accuracy_dt:.2f}")


#### K-NN


In [ ]:
# Initialisation d'un classificateur K-Nearest Neighbors (KNN).
knn_classifier = KNeighborsClassifier()

# Entraînement du KNN sur les données d'entraînement standardisées.
knn_classifier.fit(X_train_scaled, y_train)

# Prédiction des étiquettes pour les données de test standardisées.
y_pred_knn = knn_classifier.predict(X_test_scaled)

# Calcul de la précision du KNN sur l'ensemble de test.
accuracy_knn = accuracy_score(y_test, y_pred_knn)

# Affichage de la précision obtenue.
print(f"Accuracy du KNN : {accuracy_knn:.2f}")


#### Random Forest


In [ ]:
# Initialisation d'un classificateur Random Forest.
rf_classifier = RandomForestClassifier()

# Entraînement du Random Forest sur les données d'entraînement (non standardisées).
rf_classifier.fit(X_train, y_train)

# Prédiction des étiquettes pour les données de test.
y_pred_rf = rf_classifier.predict(X_test)

# Calcul de la précision du Random Forest sur l'ensemble de test.
accuracy_rf = accuracy_score(y_test, y_pred_rf)

# Affichage de la précision obtenue.
print(f"Accuracy du Random Forest : {accuracy_rf:.2f}")


#### Regression Logistique


In [ ]:
# Initialisation d'un classificateur de Régression Logistique avec un nombre maximum d'itérations fixé à 1000.
lr_classifier = LogisticRegression(max_iter=1000)

# Entraînement de la Régression Logistique sur les données d'entraînement standardisées.
lr_classifier.fit(X_train_scaled, y_train)

# Prédiction des étiquettes pour les données de test standardisées.
y_pred_lr = lr_classifier.predict(X_test_scaled)

# Calcul de la précision de la Régression Logistique sur l'ensemble de test.
accuracy_lr = accuracy_score(y_test, y_pred_lr)

# Affichage de la précision obtenue.
print(f"Accuracy de la Régression Logistique : {accuracy_lr:.2f}")


#### Resultats


In [ ]:
# Affichage des résultats finaux pour tous les modèles entraînés.
print("\nRésultats des modèles :\n")
print(f"Accuracy du Random Forest : {accuracy_rf:.2f}")
print(f"Accuracy du SVM : {accuracy_svm:.2f}")
print(f"Accuracy de la Régression Logistique : {accuracy_lr:.2f}")
print(f"Accuracy du KNN : {accuracy_knn:.2f}")
print(f"Accuracy de l'Arbre de Décision : {accuracy_dt:.2f}")


##### **INTERESTING TO LOOK AT BECAUSE RANDOM FOREST IS THE BEST ONE**


In [ ]:
# Affichage du rapport de classification pour le modèle Random Forest.
print("\nRapport de classification pour le Random Forest :")
print(classification_report(y_test, y_pred_rf, target_names=label_encoder.classes_))
# Le rapport inclut des métriques comme la précision, le rappel, et le F1-score pour chaque classe.

# Affichage de la matrice de confusion pour le modèle Random Forest.
print("\n\n\nMatrice de confusion pour le Random Forest :")
print(confusion_matrix(y_test, y_pred_rf))
# La matrice de confusion montre le nombre de prédictions correctes et erronées pour chaque classe.


In [ ]:
# Validation croisée pour le Random Forest
from sklearn.model_selection import cross_val_score

# Calcule les scores de précision pour chaque fold
scores = cross_val_score(rf_classifier, X, y_encoded, cv=5)
# Affiche les scores de chaque fold
print(f"Scores de validation croisée (Random Forest) : {scores}")
# Affiche la moyenne des scores, indicateur global de performance
print(f"Score moyen : {scores.mean():.2f}")


La case suivante prend 8 minutes à elle seule pour tourner, les résultats que j'ai obtenu sont les suivants :

- Meilleurs paramètres : {'max_depth': 20, 'min_samples_split': 10, 'n_estimators': 200}
- Meilleure score : 0.56


In [ ]:
# # Recherche des meilleurs hyperparamètres pour le Random Forest à l'aide de GridSearchCV.
# from sklearn.model_selection import GridSearchCV

# # Définition de la grille d'hyperparamètres à tester.
# param_grid = {
#     'n_estimators': [50, 100, 200],  # Nombre d'arbres dans la forêt
#     'max_depth': [None, 5, 10, 20],  # Profondeur maximale des arbres
#     'min_samples_split': [2, 5, 10]  # Nombre minimum d'échantillons pour diviser un nœud
# }

# # Initialisation de GridSearchCV avec validation croisée (5 folds).
# grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5)

# # Entraînement et recherche des meilleurs hyperparamètres.
# grid_search.fit(X_train, y_train)

# # Affichage des meilleurs paramètres trouvés et du meilleur score moyen obtenu.
# print(f"Meilleurs paramètres : {grid_search.best_params_}")  # Hyperparamètres optimaux
# print(f"Meilleur score : {grid_search.best_score_:.2f}")  # Meilleure précision moyenne sur les folds


In [ ]:
# Calcul et affichage de l'importance des caractéristiques pour le modèle Random Forest.
# Importance des caractéristiques calculée par le modèle
importances = rf_classifier.feature_importances_
# Tri des indices des caractéristiques par ordre décroissant d'importance
indices = np.argsort(importances)[::-1]

print("Importance des caractéristiques :")
# Parcours des caractéristiques en fonction de leur importance
for f in range(X.shape[1]):
    # Affiche le rang, le nom de la caractéristique (tiré de quantitative_columns), et son importance
    print(
        f"{f + 1}. {quantitative_columns[indices[f]]} ({importances[indices[f]]:.4f})"
    )
